In [2]:
from web_scraper.web_scraper import CompendiumScraper
import pandas as pd

# Running To-Do List for Scraper
- Account for Jack Garland's last followup HP being uncapped
- Account for Sherlotta's first LD HP attack being uncapped
- Account for Sherlotta's first Chuck Staff HP attack being uncapped. The splash to other targets deals exactly half of the first HP attack (maybe it could be boosted to full? unsure).
- Add something character-level that indicates whether they have a BT or not (this will be needed for when we indicate whether a character's BT is active)
- Add something that will pull character images from their profile and save them in a folder for later use (By.XPATH, "//img[@class='charmanimage']").get_attribute("src")

In [3]:
cs = CompendiumScraper()

Error getting version of chromedriver 115. Retrying with chromedriver 114 (attempt 1/5)


In [6]:
jg_df = cs.extract_ability_hp_attack_count('jackgarland')

In [7]:
jg_df

,char_name,ability_name,main_target_hp_attacks,non_target_hp_attacks,hp_dmg_cap_up_perc
0,jackgarland,HP Attack+,1,0,0
1,jackgarland,HP Attack++,1,0,0
2,jackgarland,Spinning Slash,3,3,20
3,jackgarland,Soul Burst,2,0,0
4,jackgarland,Highbringer,3,1,20
5,jackgarland,Lightbringer,5,5,10
6,jackgarland,Zantetsuken,3,3,15
7,jackgarland,Soul Burst+,3,0,0
8,jackgarland,Soul of Chaos,2,2,400
9,jackgarland,Thirst for Chaos,3,3,250


In [9]:
test_dict = {
    'test_one': 1,
    'test_two':2,
    'test_three':3
}